In [1]:
import sys
sys.path.append('../')
from model.optimization_model import OptimizationModel
import pyomo.environ as pyo
import pandas as pd

In [2]:
df = pd.read_csv('../data/consumo_material_preprocessed.csv')

In [3]:
df = df[df['FECHAPEDIDO'].str.contains('2023')]
df = df.reset_index(drop=True)

CODIGO_PROD = 65056
data = df[df['CODIGO_NUM'] == CODIGO_PROD]
data.reset_index(drop=True, inplace=True)

In [4]:
model = OptimizationModel(data, 1, 5, 1e5, 5)

In [6]:
solver_path = '/opt/homebrew/Cellar/glpk/5.0/bin/glpsole'
opt = pyo.SolverFactory('glpk', executable=solver_path)
results = opt.solve(model)
print(f'Feasible: {results.solver.termination_condition == pyo.TerminationCondition.optimal}')
optimal_values = [pyo.value(model.p[i]) for i in model.p]
print(f'Optimal values: {optimal_values}')

executable for solver glpk. File with
name=/opt/homebrew/Cellar/glpk/5.0/bin/glpsole either does not exist or it is
not executable. To skip this validation, call set_executable with
validate=False.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyomo/opt/base/solvers.py", line 151, in __call__
    opt = self._cls[_name](**kwds)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 91, in __init__
    SystemCallSolver.__init__(self, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyomo/opt/solver/shellcmd.py", line 67, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyomo/opt/solver/shellcmd.py", line 116, in set_executable
    raise ValueEr

ApplicationError: Solver (_glpk_shell) not available

In [ ]:
import math
# print 12 choose 6
print(math.factorial(12)/(math.factorial(6)*math.factorial(12-6)))

924.0
